In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
import random
import pickle
import itertools
import math
import urllib.request as urllib
import io
import zipfile

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import locally_linear_embedding
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.linalg import diag
from tensorflow.keras import callbacks
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
from networkx.generators.community import LFR_benchmark_graph
from itertools import count
from scipy.spatial import distance_matrix

from helpers import *
from Autoencoder import *

In [2]:
cs = pickle.load(open('graphs/citeseer.p', 'rb'))

In [3]:
X_cs_features = create_feature_matrix(cs, 3703)

In [4]:
X_cos_sim = cosine_similarity(X_cs_features)

In [5]:
X_cos_sim = top_k(X_cos_sim, int(average_community_size(cora_labels(cs))))

In [6]:
#3312-512-256-128-64

A = create_adjacency_matrix(cs)

#modularity
X_modularity = tf.convert_to_tensor(nx.modularity_matrix(cs).astype('float32'))

#markov
X_markov = tf.convert_to_tensor(markov_matrix(A, X_cos_sim).astype('float32'))

#similarity (Sørensen–Dice) S
X_similarity = tf.convert_to_tensor(adjacency_to_similarity(A).astype('float32'))

#probability transition T
X_prob_trans = tf.convert_to_tensor(probability_transition_matrix(A, 4))

In [7]:
X_cs_1 = X_modularity

In [8]:
#train new network

ae_cs_1 = Autoencoder(3312, 512, k_reg=tf.keras.regularizers.L2(), act_reg=SparseRegularizer())
history_cs_1 = train(ae_cs_1, 5000, 3312, X_modularity)

In [11]:
history_cs_2 = train(ae_cs_1, 5000, 3312, X_similarity)

In [29]:
history_cs_3 = train(ae_cs_1, 5000, 3312, X_markov)

In [38]:
history_cs_4 = train(ae_cs_1, 5000, 3312, X_prob_trans)

In [48]:
H_cs = ae_cs_1.encoder(X_prob_trans)
labels = cora_labels(cs)
kmeans_cs = KMeans(n_clusters=6, random_state=0).fit(H_cs)

In [49]:
normalized_mutual_info_score(labels, kmeans_cs.labels_)

0.3720536364702456